In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import analysis
import recording_selection
import db
import fetcher
from recordings import Recording, RecordingOverrides
from species import CommonName
from trim_recordings import detect_utterances

import IPython
import librosa
import librosa.display
import librosa.feature
import matplotlib
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import numpy as np
import pydub
import scipy.ndimage
from tqdm import tqdm

import collections
import hashlib
import io
import itertools
import logging
import multiprocessing
import warnings

warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
plt.rcParams['svg.fonttype'] = 'none'
# logging.getLogger().setLevel(level=logging.DEBUG)

In [ ]:
session = db.create_session('master.db')
recordings_fetcher = fetcher.Fetcher('recordings', pool_size=8)
recording_overrides = RecordingOverrides()

Load recordings from the database and filter them according to some selection criteria:
right species, contains the data we need, good quality, not too short and not too long.

Then get them from the cache or download them if necessary.

In [ ]:
DUTCH_COMMON_NAME = 'Merel'
COUNT = 12

species = session.query(CommonName).filter(
    CommonName.language_code == 'nl',
    CommonName.common_name == DUTCH_COMMON_NAME,
).one().species

selection = recording_selection.RecordingSelection(species, session, recordings_fetcher, recording_overrides)
print(f'Found {len(selection.candidate_recordings)} candidate recordings; loading {COUNT} suitable ones...')

analyses = list(itertools.islice(selection.suitable_recordings(), COUNT))

for a in analyses:
    fig, ax = plt.subplots(figsize=(9, 2))
    librosa.display.specshow(librosa.power_to_db(a.mel_spectrogram),
                             x_axis='time', y_axis='mel', fmin=0.0, fmax=analysis.SAMPLE_RATE / 2,
                             cmap='magma', vmin=-80, vmax=0,
                             hop_length=analysis.FFT_HOP_LENGTH, sr=analysis.SAMPLE_RATE, ax=ax)
    ax.set_title(f'{a.recording.recording_id} - {a.recording.type}')
    for voc in a.vocalizations:
        ax.add_patch(
            Rectangle((voc.start, 0), voc.end - voc.start, analysis.SAMPLE_RATE / 2,
                      edgecolor='none', facecolor='#00ff0050'))
    plt.show()

    IPython.display.display(IPython.display.Audio(a.sound, rate=analysis.SAMPLE_RATE))

In [ ]:
vocalizations = []
for a in analyses:
    vocalizations.extend(a.vocalizations)
print(f'Found {len(vocalizations)} vocalizations')

In [ ]:
fig, ax = plt.subplots(8, (len(vocalizations) + 7) // 8, figsize=(15, 10))
for voc, ax in zip(vocalizations, ax.flat):
    ax.imshow(librosa.power_to_db(voc.filtered_mel_spectrogram), cmap='magma', aspect='auto', origin='lower')
    ax.set_xticks([])
    ax.set_yticks([])